# Create weights for OM4p25

In [1]:
import xarray as xr
import xesmf
import numpy as np

In [2]:
erainterim_indir = '/Volumes/P4/workdir/raphael/ERAinterim/nc_processed/'

In [3]:
erainterim_outdir = '/Volumes/P4/workdir/raphael/ERAinterim/nc_processed_OM4p25/'

## Building the grids

In [2]:
# ERA-interim
erai_grid = xr.open_dataset('/Volumes/L4/ERAinterim/erai_grid.nc')
#erai_grid = xr.open_dataset(f'{erainterim_indir}/precip_ERAinterim_1979_daily_ROMS.nc', decode_times=False, drop_variables=['rain', 'rain_time'])

lon = erai_grid['lon'].values
lon_bnds = np.concatenate((np.array([lon[0] -0.5 * 0.7031]), 0.5 * (lon[:-1] + lon[1:]), np.array([lon[-1] + 0.5 * 0.7031])), axis=0)

lat = erai_grid['lat'].values
lat_bnds = np.concatenate((np.array([-90]), 0.5 * (lat[:-1] + lat[1:]), np.array([90])), axis=0)

erai_grid['lon_b'] = xr.DataArray(data=lon_bnds, dims=('lonp1'))
erai_grid['lat_b'] = xr.DataArray(data=lat_bnds, dims=('latp1'))

In [3]:
erai_grid

<xarray.Dataset>
Dimensions:  (lat: 256, latp1: 257, lon: 512, lonp1: 513)
Coordinates:
  * lon      (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
  * lat      (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
Dimensions without coordinates: latp1, lonp1
Data variables:
    lon_b    (lonp1) float64 -0.3515 0.3516 1.055 1.758 ... 358.2 358.9 359.6
    lat_b    (latp1) float64 -90.0 -89.11 -88.42 -87.72 ... 88.42 89.11 90.0

In [4]:
# OM4p25
#om4_supergrid = xr.open_dataset('/Volumes/P4/workdir/raphael/OM4/OM4_025/mosaic.v20170622.unpacked/ocean_hgrid.nc')
om4_supergrid = xr.open_dataset('ocean_hgrid.nc')

In [5]:
om4_supergrid

<xarray.Dataset>
Dimensions:   (nx: 2880, nxp: 2881, ny: 2160, nyp: 2161)
Dimensions without coordinates: nx, nxp, ny, nyp
Data variables:
    angle_dx  (nyp, nxp) float64 ...
    area      (ny, nx) float64 ...
    dx        (nyp, nx) float64 ...
    dy        (ny, nxp) float64 ...
    tile      |S255 ...
    x         (nyp, nxp) float64 ...
    y         (nyp, nxp) float64 ...

In [6]:
om4_grid = xr.Dataset()
lon = om4_supergrid['x'].values[1::2,1::2].copy()
lat = om4_supergrid['y'].values[1::2,1::2].copy()
lon_b = om4_supergrid['x'].values[0::2,0::2].copy()
lat_b = om4_supergrid['y'].values[0::2,0::2].copy()

om4_grid['lon'] = xr.DataArray(data=lon, dims=('y', 'x'))
om4_grid['lat'] = xr.DataArray(data=lat, dims=('y', 'x'))
om4_grid['lon_b'] = xr.DataArray(data=lon_b, dims=('yp1', 'xp1'))
om4_grid['lat_b'] = xr.DataArray(data=lat_b, dims=('yp1', 'xp1'))

In [7]:
om4_grid

<xarray.Dataset>
Dimensions:  (x: 1440, xp1: 1441, y: 1080, yp1: 1081)
Dimensions without coordinates: x, xp1, y, yp1
Data variables:
    lon      (y, x) float64 -299.7 -299.5 -299.2 -299.0 ... 60.0 60.0 60.0 60.0
    lat      (y, x) float64 -79.81 -79.81 -79.8 -79.8 ... 64.33 64.22 64.11
    lon_b    (yp1, xp1) float64 -299.8 -299.6 -299.3 -299.1 ... 60.0 60.0 60.0
    lat_b    (yp1, xp1) float64 -79.84 -79.84 -79.84 ... 64.28 64.17 64.06

In [8]:
%time regrid_conserve = xesmf.Regridder(erai_grid, om4_grid, method='conservative', periodic=True)

Create weight file: conservative_256x512_1080x1440.nc
CPU times: user 59.4 s, sys: 2.66 s, total: 1min 2s
Wall time: 1min 2s


In [9]:
%time regrid_patch = xesmf.Regridder(erai_grid, om4_grid, method='patch', periodic=True)

Create weight file: patch_256x512_1080x1440_peri.nc
CPU times: user 3min 42s, sys: 21.7 s, total: 4min 4s
Wall time: 4min 7s


In [10]:
%time regrid_bilin = xesmf.Regridder(erai_grid, om4_grid, method='bilinear', periodic=True)

Create weight file: bilinear_256x512_1080x1440_peri.nc
CPU times: user 37 s, sys: 3.4 s, total: 40.4 s
Wall time: 40.8 s
